In [1]:
import SimpleITK as sitk
from radiomics import featureextractor
import nibabel as nib
import numpy as np
from PIL import Image
from pathlib import Path
import os
import matplotlib.pyplot as plt
import csv
import pandas as pd

In [22]:
# Paths relativos de las imágenes y las máscaras
dir_datos = Path("/Users/celiaalme/Desktop/TFM/Practica_Celia/Estudios")
dir_datosROIs = Path("/Users/celiaalme/Desktop/TFM/Practica_Celia/ROIS")


params = {
    'setting': {
        'binWidth': 25,
        'label': 1,  # como la máscara es binaria, usamos el valor 1
        'voxelArrayShift': 1000  
    },
    'featureClass': {
        'shape2d': []  
    },
    'inputImageType': {
        'Original': {}  
    }
}


# Creamos el extractor de características
extractor = featureextractor.RadiomicsFeatureExtractor(**params)
extractor.enableFeatureClassByName('shape2D')

# Función para comprobar que cogeremos la máscara que se corresponda con la imagen 
# (comprobación del nombre de los directorios)
def verificar_codigo(dir_img, dir_ROIs):
    codigo = dir_ROIs[:7]
    if codigo in dir_img:
        return True
    else:
        return False  

# Iteramos para acceder a cada carpeta de los estudios

# dir: control 10 semanas, control 20 semanas, HFD 10 semanas, HFD 20 semanas
for dir in os.listdir(dir_datos):
    if not dir.startswith('.'):
        # Creamos una carpeta de CSVs en cada estudio
        dir_name = 'CSV_N4ITK'
        target_dir_name = dir_datos.joinpath(dir).joinpath(dir_name)
        if not os.path.exists(target_dir_name):
            try:
                os.makedirs(target_dir_name)
            except FileExistsError:
                pass
       
        # Creamos dos carpetas por cada carpeta CSV: machos, hembras
        subDir_name_M = 'Machos'
        subDir_name_H = 'Hembras'
        target_subDir_name_M = target_dir_name.joinpath(subDir_name_M)
        target_subDir_name_H = target_dir_name.joinpath(subDir_name_H)
        if not os.path.exists(target_subDir_name_M) and not os.path.exists(target_subDir_name_H):
            try:
                os.makedirs(target_subDir_name_H)
                os.makedirs(target_subDir_name_M)
            except FileExistsError:
                pass
                
        
        # subDir: hembras, machos
        for subDir in os.listdir(dir_datos.joinpath(dir)):
            if not subDir.startswith('.'):
                
                # resoDir: resomapper_output
                for resoDir in os.listdir(dir_datos.joinpath(dir).joinpath(subDir)):
                    if not resoDir.startswith('.'):
                        
                    # sourcedataDir: derivatives, sourcedata
                        for sourcedataDir in os.listdir(dir_datos.joinpath(dir).joinpath(subDir).joinpath(resoDir)):
                            if not sourcedataDir.startswith('.') and sourcedataDir == "sourcedata":
                                
                                # studyDir: estudio
                                for studyDir in os.listdir(dir_datos.joinpath(dir).joinpath(subDir).joinpath(resoDir).joinpath(sourcedataDir)):
                                    if not studyDir.startswith('.'):
                                        for imgfile in os.listdir(dir_datos.joinpath(dir).joinpath(subDir).joinpath(resoDir).joinpath(sourcedataDir).joinpath(studyDir)):
                                            if not imgfile.startswith('.') and imgfile.endswith('.nii.gz'):
                                                # Cargamos la imagen y las labels (no hago bucle porque hay distintos tipos de corte); cambiamos el nombre del archivo 
                                                image = sitk.ReadImage(str(dir_datos.joinpath(dir).joinpath(subDir).joinpath(resoDir).joinpath(sourcedataDir).joinpath(studyDir).joinpath(imgfile)))
                                                # Seleccionar el índice del slice deseado (el corte 3 es el que mejor se ve)
                                                slice_index = 2

                                                ex_slicer = sitk.ExtractImageFilter()
                                                ex_slicer.SetSize([image.GetWidth(), image.GetHeight(), 0])  # Z=0 → 2D real
                                                ex_slicer.SetIndex([0, 0, slice_index])
                                                
                                                slice = ex_slicer.Execute(image)
                                                
                                                # Ajustamos el espaciado de la imagen 2D
                                                slice = sitk.Cast(slice, sitk.sitkFloat32)
                                                
                                                
                                                # Fijamos origen, spacing y dirección 2D
                                                slice.SetOrigin([0.0, 0.0])
                                                slice.SetSpacing([1.0, 1.0])
                                                slice.SetDirection([1.0, 0.0, 0.0, 1.0])

                                                #print("Slice:", slice.GetSize(), slice.GetSpacing(), slice.GetDirection())
                                                
                                                for dir_ROI in os.listdir(dir_datosROIs):
                                                    if not dir_ROI.startswith('.'):
                                                        for subDir_ROI in os.listdir(dir_datosROIs.joinpath(dir_ROI)):
                                                            if not subDir_ROI.startswith('.'):
                                                                for studyDir_ROI in os.listdir(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI)):
                                                                    if not studyDir_ROI.startswith('.'):
                                                                        if verificar_codigo(imgfile, studyDir_ROI):
                                                                            
                                                                            # Paso clave: llevamos a cabo el preprocesado de las imágenes:
                                                                            # esto nos sirve para evitar que las diferencias de intensidad entre imágenes afecten a la extracción de features
                                                                                                                                                                                                                
                                                                                
                                                                            # Creamos diccionario para guardar las features de cada ROI y pasarlo a CSV
                                                                            results = {}
                                                                            
                                                                            for maskfile in os.listdir(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI).joinpath(studyDir_ROI)):
                                                                                # Seleccionamos las máscaras en archivo NIfTI
                                                                                if maskfile.endswith('.nii'): 
                                                                                    mask_2D = sitk.ReadImage(str(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI).joinpath(studyDir_ROI).joinpath(maskfile)))
                                                                                    mask_2D = sitk.Cast(mask_2D, sitk.sitkUInt8)

                                                                                    # Si la máscara es 3D, convertimos a 2D real
                                                                                    if mask_2D.GetDimension() == 3:
                                                                                        mask_2D = sitk.Extract(mask_2D, size=[mask_2D.GetWidth(), mask_2D.GetHeight(), 0], index=[0, 0, 0])

                                                                                    # Alineanos máscara con el slice
                                                                                    mask_2D.SetOrigin(slice.GetOrigin())
                                                                                    mask_2D.SetSpacing(slice.GetSpacing())
                                                                                    mask_2D.SetDirection(slice.GetDirection())
                                                                                    
                                                                                    # Creamos máscara binaria para N4                                                                                    
                                                                                    mask_N4 = sitk.BinaryThreshold(mask_2D, 1, 255, 1, 0)

                                                                                    roi_name = maskfile.replace('.nii', '').replace('mask_', '')

                                                                                    # N4 Bias Field Correction                                                                                    
                                                                                    slice_normalized = sitk.N4BiasFieldCorrection(slice, mask_2D)

                                                                                    mask_array = sitk.GetArrayFromImage(mask_N4)
                                                                                    n_voxels = np.sum(mask_array > 0)
                                                                                    
                                                                                    # Comprobamos que la ROI no sea demasiado pequeña para extraer features
                                                                                    if n_voxels < 2:
                                                                                        print(f" ROI {roi_name}{maskfile} muy pequeña!")
                                                                                        features = {'diagnostico': 'ROI_invalida'}
                                                                                    else:
                                                                                        features = extractor.execute(slice_normalized, mask_N4)
    
                                                                                    
                                                                                    results[roi_name] = features


                                                                            results_df = pd.DataFrame(results)
                                                                            csv_filename = f"{studyDir_ROI}.csv"
                                                                            if 'HEMBRAS' in str(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI)):                                                             
                                                                                results_df.to_csv(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI).joinpath(target_subDir_name_H).joinpath(csv_filename))
                                                                            if 'MACHOS' in str(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI)):                                                             
                                                                                results_df.to_csv(dir_datosROIs.joinpath(dir_ROI).joinpath(subDir_ROI).joinpath(target_subDir_name_M).joinpath(csv_filename))                                                                   
                                                                        

Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is 